## 1. Import dependencies

In [102]:
import pandas as pd
# !pip install openpyxl
import openpyxl
from sqlalchemy import create_engine
from db_conn import DB_conn

## 2. Retrieve Data from the excel file

In [103]:
excel_file = 'Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2022.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)
sheets = all_sheets.keys()

for sheet_name in sheets:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    sheet.to_csv("%s.csv" % sheet_name, index=False)

## Loop thru excel file to convert worksheets to csv files

In [104]:
#Create list of dataframes from csv files
file_list = []
for i in range(1,7):
    file_list.append(f"Table 0{i}.csv")

df_list = [pd.read_csv(file) for file in file_list]
file_list

['Table 01.csv',
 'Table 02.csv',
 'Table 03.csv',
 'Table 04.csv',
 'Table 05.csv',
 'Table 06.csv']

In [112]:
postcode_file = 'au_postcodes.csv'
postcode_geo_df = pd.read_csv(postcode_file)
print(f"All Aus. postcodes: {postcode_geo_df.shape}")

postcode_geo_df = postcode_geo_df.loc[postcode_geo_df.state_code == 'VIC']
print(f"Only Vic. postcodes: {postcode_geo_df.shape}")
postcode_geo_df.head()

All Aus. postcodes: (16875, 7)
Only Vic. postcodes: (3260, 7)


,postcode,place_name,state_name,state_code,latitude,longitude,accuracy
11712,3000,Melbourne,Victoria,VIC,-37.8140,144.9633,4.0
11713,3001,Melbourne,Victoria,VIC,-37.8140,144.9633,4.0
11714,3002,East Melbourne,Victoria,VIC,-37.8167,144.9879,4.0
11715,3003,West Melbourne,Victoria,VIC,-37.8101,144.9500,4.0
11716,3004,Melbourne,Victoria,VIC,-37.8140,144.9633,4.0


In [22]:
postcode_geo_df.dtypes

postcode        int64
place_name     object
state_name     object
state_code     object
latitude      float64
longitude     float64
accuracy      float64
dtype: object

### Create police_region_df from Table1 which is high level data, starts from Police Region
(PR, a police regions have a one to many relationshgip with Police Service Area and local Government Areas )

In [105]:
# Exract rows that are greater than year 2018 (use 4 years data, 2019-2022) 
# and drop column 'Year ending' as the crime tables are created on March (no date details) every year.
police_region_df = df_list[0].loc[df_list[0].Year > 2018].drop('Year ending',axis=1)
police_region_df["Local Government Area"] = police_region_df["Local Government Area"].str.strip()

# Drop Total(summary) rows
print(f"Before Total rows drop: {police_region_df.shape}")
police_region_df = police_region_df.drop(police_region_df[(police_region_df['Local Government Area'] == 'Total')].index)
print(f"After Total rows drop: {police_region_df.shape}")
police_region_df.head()


Before Total rows drop: (348, 5)
After Total rows drop: (324, 5)


,Year,Police Region,Local Government Area,Incidents Recorded,"Rate per 100,000 population"
0,2022,1 North West Metro,Banyule,5244,4086.865525
1,2022,1 North West Metro,Brimbank,12330,6200.959480
2,2022,1 North West Metro,Darebin,9276,5751.475727
3,2022,1 North West Metro,Hobsons Bay,4599,4804.896128
4,2022,1 North West Metro,Hume,12196,4919.684615


In [24]:
# police_region_df.dtypes

### Create police_division_df from Table2 which is in Police Service Area (PSA) and Local Government Area (LGA), breaks down to different crime/offence and subtype types

In [113]:
# Due to Heroku free data capacity allowance constraint, we use data rows of year 2022.
police_division_df = df_list[1].loc[df_list[1].Year > 2021].drop('Year ending',axis=1)
print(f"police_division_df shape: {police_division_df.shape}")
police_division_df.head()

police_division_df shape: (5017, 9)


,Year,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population"
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137


In [114]:
police_division_df.dtypes 


Year                                 int64
Police Service Area                 object
Local Government Area               object
Offence Division                    object
Offence Subdivision                 object
Offence Subgroup                    object
Incidents Recorded                   int64
PSA Rate per 100,000 population    float64
LGA Rate per 100,000 population    float64
dtype: object

#### Take the FIRST postcode for each LGA, where one LGA can have multiple suburbs, each suburb can have its own postcode

In [115]:
region_postcode_df = df_list[2].groupby(by=['Local Government Area','Postcode'], as_index=False).first()
region_postcode_df=region_postcode_df.rename(columns = {'Postcode' : 'postcode'})
print(f"region_postcode_df: {region_postcode_df.shape}")
region_postcode_df.head()


region_postcode_df: (971, 9)


,Local Government Area,postcode,Year,Year ending,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,Alpine,3691,2022,March,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2
1,Alpine,3697,2022,March,Tawonga,B Property and deception offences,B40 Theft,B49 Other theft,1
2,Alpine,3698,2022,March,Tawonga South,B Property and deception offences,B20 Property damage,B21 Criminal damage,1
3,Alpine,3699,2022,March,Mount Beauty,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,4
4,Alpine,3737,2022,March,Barwidgee,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,2


In [44]:
region_postcode_df.dtypes

Local Government Area    object
postcode                  int64
Year                      int64
Year ending              object
Suburb/Town Name         object
Offence Division         object
Offence Subdivision      object
Offence Subgroup         object
Incidents Recorded        int64
dtype: object

In [116]:
region_postcode_df = region_postcode_df.drop(['Year','Year ending', 'Suburb/Town Name', 'Offence Division', 'Offence Subdivision', 'Offence Subgroup', 'Incidents Recorded'], axis=1)
region_postcode_df.sort_values(by='Local Government Area' , ascending=True, inplace=True);
print(region_postcode_df.shape)
region_postcode_df.head()

(971, 2)


,Local Government Area,postcode
0,Alpine,3691
1,Alpine,3697
2,Alpine,3698
3,Alpine,3699
4,Alpine,3737


In [117]:
df = pd.merge(region_postcode_df,postcode_geo_df,how="left", on="postcode")
df = df.groupby(by=['Local Government Area'], as_index=False).first()
df = df.drop(['state_code', 'state_name', 'accuracy'], axis = 1)
region_postcode_df = df
print(f"region_postcode_df : {region_postcode_df.shape}")
region_postcode_df.head()

region_postcode_df : (79, 5)


,Local Government Area,postcode,place_name,latitude,longitude
0,Alpine,3691,Killara,-36.1396,146.9514
1,Ararat,3469,Nowhere Creek,-37.1389,143.2611
2,Ballarat,3357,Buninyong,-37.6502,143.8839
3,Banyule,3095,Research,-37.7000,145.1833
4,Bass Coast,3990,Glen Forbes,-38.4500,145.5333


### Add Police Region column to create a more complete dataset to enable consistent data visualisation 

In [118]:
police_region2022_df = ''
police_region2022_df = df_list[0].loc[df_list[0].Year > 2021].drop('Year ending',axis=1)
police_region2022_df["Local Government Area"] = police_region2022_df["Local Government Area"].str.strip() # must strip off the first blank character
print(f"police_region2022_df shape: {police_region2022_df.shape}")

police_region2022_df shape: (87, 5)


In [119]:
police_division_df=pd.merge(police_division_df, police_region2022_df[["Police Region","Local Government Area"]],
           how="left", on="Local Government Area")
police_division_df=police_division_df[police_division_df["Year"]> 2021]
print(f"police_division_df shape: {police_division_df.shape}")
police_division_df.head()     

police_division_df shape: (5017, 10)


,Year,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population",Police Region
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089,4 Western
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128,4 Western
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807,4 Western
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740,4 Western
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137,4 Western


### Create offence_summary_df for Local Government Region base on different crime types

In [120]:
offence_summary_df = df_list[5]
offence_summary_df = offence_summary_df.drop('(blank)',axis=1)
offence_summary_df.head()

,Year,Local Government Area,A Crimes against the person,B Property and deception offences,C Drug offences,D Public order and security offences,E Justice procedures offences,F Other offences
0,2019,Alpine,114,161,26.0,48,50,0
1,2019,Ararat,223,376,65.0,81,165,2
2,2019,Ballarat,1372,5061,204.0,372,1105,15
3,2019,Banyule,957,4063,223.0,309,1442,11
4,2019,Bass Coast,376,992,78.0,119,216,2


In [34]:
offence_summary_df.dtypes 

Year                                      int64
Local Government Area                    object
A Crimes against the person               int64
B Property and deception offences         int64
C Drug offences                         float64
D Public order and security offences      int64
E Justice procedures offences             int64
F Other offences                          int64
dtype: object

In [121]:
# merge dfs to add police region, and add column offence_total
offence_summary_df=pd.merge(offence_summary_df, police_region2022_df[["Police Region","Local Government Area"]],
           how="left", on="Local Government Area")
df1 = offence_summary_df
df1['offence_total'] = offence_summary_df[['A Crimes against the person','B Property and deception offences', 
              'C Drug offences','D Public order and security offences',
              'E Justice procedures offences','F Other offences']].sum(axis = 1)
offence_summary_df = df1
print(f"offence_summary_df shape: {offence_summary_df.shape}")
offence_summary_df.head()     

offence_summary_df shape: (316, 10)


,Year,Local Government Area,A Crimes against the person,B Property and deception offences,C Drug offences,D Public order and security offences,E Justice procedures offences,F Other offences,Police Region,offence_total
0,2019,Alpine,114,161,26.0,48,50,0,2 Eastern,399.0
1,2019,Ararat,223,376,65.0,81,165,2,4 Western,912.0
2,2019,Ballarat,1372,5061,204.0,372,1105,15,4 Western,8129.0
3,2019,Banyule,957,4063,223.0,309,1442,11,1 North West Metro,7005.0
4,2019,Bass Coast,376,992,78.0,119,216,2,2 Eastern,1783.0


## 3. Rename data frame columns to have more unified names

In [122]:
police_region_df.rename(columns = {'Year' : 'year', 'Police Region' : 'region_name', 
            'Local Government Area' : 'lga_name', 'Incidents Recorded' : 'incident_count',
            'Rate per 100,000 population' : 'rate_per_100000pop'}, inplace = True)  
police_region_df.head()         

,year,region_name,lga_name,incident_count,rate_per_100000pop
0,2022,1 North West Metro,Banyule,5244,4086.865525
1,2022,1 North West Metro,Brimbank,12330,6200.959480
2,2022,1 North West Metro,Darebin,9276,5751.475727
3,2022,1 North West Metro,Hobsons Bay,4599,4804.896128
4,2022,1 North West Metro,Hume,12196,4919.684615


In [123]:
police_division_df.rename(columns = {'Year' : 'year', 'Police Service Area' : 'psa_name',
            'Local Government Area' : 'lga_name', 'Offence Division' : 'offence_div_name',
            'Offence Subdivision' : 'offence_subdiv_name', 'Offence Subgroup' : 'offence_subgrp_name',
            'Incidents Recorded' : 'incident_count', 'PSA Rate per 100,000 population' : 'psa_rate_per_100000pop',
            'LGA Rate per 100,000 population' : 'lga_rate_per_100000pop', 'Police Region' : 'region_name'}, inplace = True)
police_division_df.head() 

,year,psa_name,lga_name,offence_div_name,offence_subdiv_name,offence_subgrp_name,incident_count,psa_rate_per_100000pop,lga_rate_per_100000pop,region_name
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089,4 Western
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128,4 Western
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807,4 Western
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740,4 Western
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137,4 Western


In [124]:
region_postcode_df = region_postcode_df.rename(columns = {'Local Government Area' : 'lga_name'})
region_postcode_df.head()            

,lga_name,postcode,place_name,latitude,longitude
0,Alpine,3691,Killara,-36.1396,146.9514
1,Ararat,3469,Nowhere Creek,-37.1389,143.2611
2,Ballarat,3357,Buninyong,-37.6502,143.8839
3,Banyule,3095,Research,-37.7000,145.1833
4,Bass Coast,3990,Glen Forbes,-38.4500,145.5333


In [125]:
offence_summary_df.rename(columns = {'Year' : 'year', 'Local Government Area' : 'lga_name', 'Police Region' : 'region_name',
            'A Crimes against the person' : 'a_crime_vs_person', 'B Property and deception offences' : 'b_property_deception',
            'C Drug offences' : 'c_drug_offence', 'D Public order and security offences' : 'd_public_order_security',
            'E Justice procedures offences' : 'e_justice_offence', 'F Other offences' : 'f_other_offence'}, inplace = True)

offence_summary_df.head()            

,year,lga_name,a_crime_vs_person,b_property_deception,c_drug_offence,d_public_order_security,e_justice_offence,f_other_offence,region_name,offence_total
0,2019,Alpine,114,161,26.0,48,50,0,2 Eastern,399.0
1,2019,Ararat,223,376,65.0,81,165,2,4 Western,912.0
2,2019,Ballarat,1372,5061,204.0,372,1105,15,4 Western,8129.0
3,2019,Banyule,957,4063,223.0,309,1442,11,1 North West Metro,7005.0
4,2019,Bass Coast,376,992,78.0,119,216,2,2 Eastern,1783.0


## 4. Export dataframes to csv files for checking data quality

In [126]:
police_region_df.to_csv('outputs/police_region_df.csv',index=0)
police_division_df.to_csv('outputs/police_division_df.csv',index=0)
region_postcode_df.to_csv('outputs/region_postcode_df.csv',index=0)
offence_summary_df.to_csv('outputs/offence_summary_df.csv',index=0)

In [127]:

region_postcode_df.to_csv('outputs/region_postcode_df.csv',index=0)

## 5. Connect to database 

In [128]:
engine = create_engine (DB_conn)


In [129]:
engine.table_names()

c:\Users\youpe.DESKTOP-UVNLIE6\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['region_incident', 'lga_offence', 'lga_offence_summary']

In [130]:
police_region_df.to_sql(name='region_incident', con=engine, if_exists='replace', index=False)
print(pd.read_sql_query('select count(*) from region_incident', con=engine))
pd.read_sql_query('select * from region_incident', con=engine).head()

   count
0    324


,year,region_name,lga_name,incident_count,rate_per_100000pop
0,2022,1 North West Metro,Banyule,5244,4086.865525
1,2022,1 North West Metro,Brimbank,12330,6200.959480
2,2022,1 North West Metro,Darebin,9276,5751.475727
3,2022,1 North West Metro,Hobsons Bay,4599,4804.896128
4,2022,1 North West Metro,Hume,12196,4919.684615


In [61]:
police_division_df.to_sql(name='lga_offence', con=engine, if_exists='replace', index=False)
print(pd.read_sql_query('select count(*) from lga_offence', con=engine))
pd.read_sql_query('select * from lga_offence', con=engine).head()

   count
0   5017


,year,psa_name,lga_name,offence_div_name,offence_subdiv_name,offence_subgrp_name,incident_count,psa_rate_per_100000pop,lga_rate_per_100000pop,region_name
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089,4 Western
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128,4 Western
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807,4 Western
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740,4 Western
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137,4 Western


In [62]:
offence_summary_df.to_sql(name='lga_offence_summary', con=engine, if_exists='replace', index=False)
print(pd.read_sql_query('select count(*) from lga_offence_summary', con=engine))
pd.read_sql_query('select * from lga_offence_summary', con=engine).head()

   count
0    316


,year,lga_name,a_crime_vs_person,b_property_deception,c_drug_offence,d_public_order_security,e_justice_offence,f_other_offence,region_name,offence_total
0,2019,Alpine,114,161,26.0,48,50,0,2 Eastern,399.0
1,2019,Ararat,223,376,65.0,81,165,2,4 Western,912.0
2,2019,Ballarat,1372,5061,204.0,372,1105,15,4 Western,8129.0
3,2019,Banyule,957,4063,223.0,309,1442,11,1 North West Metro,7005.0
4,2019,Bass Coast,376,992,78.0,119,216,2,2 Eastern,1783.0


In [131]:
region_postcode_df.to_sql(name='region_postcode', con=engine, if_exists='replace', index=False)
print(pd.read_sql_query('select count(*) from region_postcode', con=engine))
pd.read_sql_query('select * from region_postcode', con=engine).head()

   count
0     79


,lga_name,postcode,place_name,latitude,longitude
0,Alpine,3691,Killara,-36.1396,146.9514
1,Ararat,3469,Nowhere Creek,-37.1389,143.2611
2,Ballarat,3357,Buninyong,-37.6502,143.8839
3,Banyule,3095,Research,-37.7000,145.1833
4,Bass Coast,3990,Glen Forbes,-38.4500,145.5333


In [132]:
# Check database table's availability
engine.table_names()

c:\Users\youpe.DESKTOP-UVNLIE6\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['region_incident', 'region_postcode', 'lga_offence', 'lga_offence_summary']